# Visualización de los datos obtenidos

Se van a contar las palabras y visualizar la ocurrencia de las mismas.



In [25]:
import pandas as pd  
import numpy as np
import matplotlib.pyplot as plt

#Importar el archivo con los tweets pre procesados (limpios)
csv = 'tweets_limpios.csv'
my_df = pd.read_csv(csv,index_col=0)
my_df.head()

#Imprimir las primeras líneas del archivo para validar una carga correcta
my_df.head()

,Texto,Polaridad
0,salgo de #veotv que día más largoooooo,NONE
1,@pauladelasheras no te libraras de ayudar meno...,NEU
2,@marodriguezb gracias mar,P
3,off pensando en el regalito sinde la que se va...,N+
4,conozco a alguien q es adicto al drama! ja ja ...,P+


## Uso de CountVectorizer
CountVectorizer convierte los comentarios en una matriz en la que cada palabra es una columna cuyo valor es el número de veces que aparece en cada tweet.

Esta parte añade procesamiento obtenido de https://new.pybonacci.org/2015/11/24/como-hacer-analisis-de-sentimiento-en-espanol-2/

In [26]:
#Descargar las stopwords y punkt primero con el downloader de nltk
#En la consola de Anaconda, correr: python -m nltk.downloader stopwords
#En la consola de Anaconda, correr: python -m nltk.downloader punkt
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.data import load
from nltk.stem import SnowballStemmer
from string import punctuation
from sklearn.feature_extraction.text import CountVectorizer

#Carga los stopwords de español y el stemmer
spanish_stopwords = stopwords.words('spanish')
stemmer = SnowballStemmer('spanish')

#Se crea la lista de puntuación para eliminar de las frecuencias
non_words = list(punctuation)
non_words.extend(['¿', '¡'])
non_words.extend(map(str,range(10)))

#Método que obtiene el lema de las palabras
def stem_tokens(tokens, stemmer):
    stemmed = []
    #Para cada palabra, obtenga el lema
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

#Método que tokeniza los tweets y le obtiene el lema a cada token
def tokenize(text):
    #Se quitan las puntuaciones
    text = ''.join([c for c in text if c not in non_words])
    #Se obtienen los tokens
    tokens =  word_tokenize(text)

    # stem
    try:
        stems = stem_tokens(tokens, stemmer)
    except Exception as e:
        print(e)
        print(text)
        stems = ['']
    return stems

#Se usa CountVectorizer para contar la frecuencia de las palabras
cvec = CountVectorizer(
                analyzer = 'word',
                tokenizer = tokenize,
                lowercase = True,
                stop_words = spanish_stopwords)
cvec.fit(my_df.Texto)
len(cvec.get_feature_names())

c:\users\joax\anaconda2\lib\site-packages\ipykernel_launcher.py:32: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal


12447

In [27]:
#Generar matrices
none_doc_matrix = cvec.transform(my_df[my_df.Polaridad == "NONE"].Texto)
pos_doc_matrix = cvec.transform(my_df[my_df.Polaridad == "P"].Texto)
neg_doc_matrix = cvec.transform(my_df[my_df.Polaridad == "N"].Texto)
posmas_doc_matrix = cvec.transform(my_df[my_df.Polaridad == "P+"].Texto)
negmas_doc_matrix = cvec.transform(my_df[my_df.Polaridad == "N+"].Texto)

none_tf = np.sum(none_doc_matrix,axis=0)
neu_tf = np.sum(neu_doc_matrix,axis=0)
neg_tf = np.sum(neg_doc_matrix,axis=0)
pos_tf = np.sum(pos_doc_matrix,axis=0)
negmas_tf = np.sum(negmas_doc_matrix,axis=0)
posmas_tf = np.sum(posmas_doc_matrix,axis=0)

none = np.squeeze(np.asarray(none_tf))
neu = np.squeeze(np.asarray(neu_tf))
neg = np.squeeze(np.asarray(neg_tf))
pos = np.squeeze(np.asarray(pos_tf))
negmas = np.squeeze(np.asarray(negmas_tf))
posmas = np.squeeze(np.asarray(posmas_tf))

term_freq_df = pd.DataFrame([none,neu,neg,pos,negmas,posmas],columns=cvec.get_feature_names()).transpose()

c:\users\joax\anaconda2\lib\site-packages\ipykernel_launcher.py:32: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal


In [28]:
#Tabular las matrices para poder ver las palabras con mayor repetición
term_freq_df.columns = ['None', 'Neutral', 'Negativo', 'Positivo', 'Negativo+', 'Positivo+']
term_freq_df['Total'] = term_freq_df['None'] + term_freq_df['Neutral']+term_freq_df['Negativo'] + term_freq_df['Positivo']+term_freq_df['Negativo+'] + term_freq_df['Positivo+']
term_freq_df.sort_values(by='Total', ascending=False).iloc[:10]

,None,Neutral,Negativo,Positivo,Negativo+,Positivo+,Total
xurlx,763,159,384,485,242,510,2543
xnumx,303,138,278,220,271,305,1515
par,141,109,194,220,179,258,1101
rt,95,40,82,150,35,200,602
q,44,72,181,63,103,99,562
tod,50,38,69,93,47,205,502
mas,43,52,96,62,86,141,480
com,58,53,104,70,83,87,455
buen,61,42,30,114,9,197,453
d,40,38,95,66,107,93,439
